In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTrajectories
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.ode_agent import ODEAgent
from cs285.agents.utils import save_leaves, load_leaves
from cs285.infrastructure import utils
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import gym
from cs285.infrastructure import pytorch_util as ptu
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax
import pickle
from tqdm import trange

In [4]:
key = jax.random.PRNGKey(0)
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [5]:
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
mpc_dt_sampler = ConstantSampler(dt=0.05)
agent_key, new_agent_key = jax.random.split(key)
neural_ode_name = "vanilla"
neural_ode_kwargs = {
    "ode_dt0": 0.005,
    "mlp_dynamics_setup": {
        "hidden_size":128,
        "num_layers":4,
        "activation":"tanh",
        "output_activation":"identity"
    }
}
optimizer_name = "adamw"
optimizer_kwargs = {"learning_rate": 1e-3}
mb_agent = ODEAgent(
    env=env,
    key=agent_key,
    neural_ode_name=neural_ode_name,
    neural_ode_kwargs=neural_ode_kwargs,
    optimizer_name=optimizer_name,
    optimizer_kwargs=optimizer_kwargs,
    ensemble_size=10,
    train_discount=1,
    mpc_horizon_steps=20,
    mpc_dt_sampler=mpc_dt_sampler,
    mpc_strategy="cem",
    mpc_discount=0.9,
    mpc_num_action_sequences=1000,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
)
replay_buffer = ReplayBufferTrajectories(seed=42)
trajs, _ = sample_n_trajectories(env, RandomPolicy(env=env), ntraj=1000, max_length=200, key=key)
replay_buffer.add_rollouts(trajs)

100%|██████████| 1000/1000 [03:17<00:00,  5.06it/s]


In [5]:
"""
with open("random_replay_buffer", "wb") as f:
    pickle.dump(replay_buffer, f)
"""

In [4]:
batch_size = 64
train_ep_len = 20
train_stride = 1
train_steps = 100

In [5]:
all_losses = []
for _ in trange(train_steps, dynamic_ncols=True):
    step_losses = []
    for i in range(mb_agent.ensemble_size):
        traj = replay_buffer.sample_rollouts(batch_size=batch_size)
        obs = utils.split_arr(np.array(traj["observations"]), length=train_ep_len, stride=train_stride)
        acs = utils.split_arr(np.array(traj["actions"]), length=train_ep_len, stride=train_stride)
        dts = utils.split_arr(np.array(traj["dts"])[..., np.newaxis], length=train_ep_len, stride=train_stride).squeeze(-1)
        batch_size, num_splitted, train_ep_len, ob_dim = obs.shape
        ac_dim = acs.shape[-1]
        obs = jnp.array(obs).reshape(batch_size * num_splitted, train_ep_len, ob_dim)
        acs = jnp.array(acs).reshape(batch_size * num_splitted, train_ep_len, ac_dim)
        times = jnp.cumsum(dts, axis=-1).reshape(batch_size * num_splitted, train_ep_len)
        loss = mb_agent.batched_update(
            i=i,
            obs=obs, 
            acs=acs, 
            times=times
        )
        step_losses.append(loss)
    all_losses.append(np.mean(step_losses))

100%|██████████| 100/100 [2:30:35<00:00, 90.36s/it]  


In [11]:
save_leaves(mb_agent, "vanilla_trained_on_100_random_steps")